## 분석목적: 모델의 input output 형태 구체화

In [1]:
import argparse
import json
import os
import sys
import time

from numba.core.errors import NumbaDeprecationWarning, NumbaPendingDeprecationWarning, NumbaWarning
import warnings
warnings.simplefilter('ignore', category=NumbaDeprecationWarning)
warnings.simplefilter('ignore', category=NumbaWarning)

import numpy as np
import torch
import yaml
from det3d.datasets import build_dataset
from det3d.models import build_detector
from det3d.torchie import Config
from det3d.torchie.apis import (
    build_optimizer,
    get_root_logger,
    init_dist,
    set_random_seed,
    train_detector,
)
import torch.distributed as dist
import subprocess

# 주피터 노트북 경로설정
os.chdir('../')


def parse_args():
    parser = argparse.ArgumentParser(description="Train a detector")
    parser.add_argument("--config", default="configs/nusc/pp/nusc_centerpoint_pp_02voxel_two_pfn_10sweep.py", help="train config file path")
    parser.add_argument("--work_dir", help="the dir to save logs and models")
    parser.add_argument("--resume_from", help="the checkpoint file to resume from")
    parser.add_argument(
        "--validate",
        action="store_true",
        help="whether to evaluate the checkpoint during training",
    )
    parser.add_argument(
        "--gpus",
        type=int,
        default=1,
        help="number of gpus to use " "(only applicable to non-distributed training)",
    )
    parser.add_argument("--seed", type=int, default=None, help="random seed")
    parser.add_argument(
        "--launcher",
        choices=["pytorch", "slurm"],
        default="pytorch",
        help="job launcher",
    )
    parser.add_argument("--local_rank", type=int, default=0)
    parser.add_argument(
        "--autoscale-lr",
        action="store_true",
        help="automatically scale lr with the number of gpus",
    )
    args = parser.parse_args(args=[])
    if "LOCAL_RANK" not in os.environ:
        os.environ["LOCAL_RANK"] = str(args.local_rank)

    return args
args = parse_args()

no apex
No Tensorflow
Deformable Convolution not built!
No APEX!


In [2]:
cfg = Config.fromfile(args.config)

In [3]:
cfg

Config (path: /home/jaelee/objdect/CenterPoint/configs/nusc/pp/nusc_centerpoint_pp_02voxel_two_pfn_10sweep.py): {'itertools': <module 'itertools' (built-in)>, 'logging': <module 'logging' from '/home/jaelee/anaconda3/envs/centerpoint/lib/python3.8/logging/__init__.py'>, 'get_downsample_factor': <function get_downsample_factor at 0x7f4cf034e550>, 'tasks': [{'num_class': 1, 'class_names': ['car']}, {'num_class': 2, 'class_names': ['truck', 'construction_vehicle']}, {'num_class': 2, 'class_names': ['bus', 'trailer']}, {'num_class': 1, 'class_names': ['barrier']}, {'num_class': 2, 'class_names': ['motorcycle', 'bicycle']}, {'num_class': 2, 'class_names': ['pedestrian', 'traffic_cone']}], 'class_names': ['car', 'truck', 'construction_vehicle', 'bus', 'trailer', 'barrier', 'motorcycle', 'bicycle', 'pedestrian', 'traffic_cone'], 'target_assigner': {'tasks': [{'num_class': 1, 'class_names': ['car']}, {'num_class': 2, 'class_names': ['truck', 'construction_vehicle']}, {'num_class': 2, 'class_na

In [4]:
# distribution 설정 안함
cfg.local_rank = args.local_rank 

In [5]:
# init logger before other steps
distributed = False
logger = get_root_logger(cfg.log_level)
logger.info("Distributed training: {}".format(distributed))
logger.info(f"torch.backends.cudnn.benchmark: {torch.backends.cudnn.benchmark}")

2024-03-19 17:48:57,910 - INFO - Distributed training: False
2024-03-19 17:48:57,910 - INFO - torch.backends.cudnn.benchmark: False


In [6]:
model = build_detector(cfg.model, train_cfg=cfg.train_cfg, test_cfg=cfg.test_cfg)

2024-03-19 17:48:57,977 - INFO - Finish RPN Initialization
2024-03-19 17:48:57,978 - INFO - num_classes: [1, 2, 2, 1, 2, 2]
2024-03-19 17:48:57,998 - INFO - Finish CenterHead Initialization


Use HM Bias:  -2.19


In [7]:
from det3d.datasets.registry import DATASETS
DATASETS
cfg.data.train['type']

'NuScenesDataset'

In [8]:
datasets = [build_dataset(cfg.data.train)]

2024-03-19 17:49:05,501 - INFO - {'car': 5, 'truck': 5, 'bus': 5, 'trailer': 5, 'construction_vehicle': 5, 'traffic_cone': 5, 'barrier': 5, 'motorcycle': 5, 'bicycle': 5, 'pedestrian': 5}
2024-03-19 17:49:05,501 - INFO - [-1]
2024-03-19 17:49:07,220 - INFO - load 62964 traffic_cone database infos
2024-03-19 17:49:07,221 - INFO - load 65262 truck database infos
2024-03-19 17:49:07,221 - INFO - load 339949 car database infos
2024-03-19 17:49:07,221 - INFO - load 161928 pedestrian database infos
2024-03-19 17:49:07,221 - INFO - load 26297 ignore database infos
2024-03-19 17:49:07,222 - INFO - load 11050 construction_vehicle database infos
2024-03-19 17:49:07,222 - INFO - load 107507 barrier database infos
2024-03-19 17:49:07,222 - INFO - load 8846 motorcycle database infos
2024-03-19 17:49:07,222 - INFO - load 8185 bicycle database infos
2024-03-19 17:49:07,222 - INFO - load 12286 bus database infos
2024-03-19 17:49:07,222 - INFO - load 19202 trailer database infos
2024-03-19 17:49:08,005

10


In [9]:
type(datasets[0].pipeline)

det3d.datasets.pipelines.compose.Compose

In [10]:
from det3d.datasets.registry import PIPELINES
PIPELINES

Registry(name=pipeline, items=['Compose', 'Reformat', 'LoadPointCloudFromFile', 'LoadPointCloudAnnotations', 'DoubleFlip', 'Preprocess', 'Voxelization', 'AssignLabel'])

In [11]:
if cfg.checkpoint_config is not None:
    # save det3d version, config file content and class names in
    # checkpoints as meta data
    cfg.checkpoint_config.meta = dict(
        config=cfg.text, CLASSES=datasets[0].CLASSES
    )


In [12]:
type(datasets[0])

det3d.datasets.nuscenes.nuscenes.NuScenesDataset

In [13]:
# add an attribute for visualization convenience
model.CLASSES = datasets[0].CLASSES
print(datasets[0].CLASSES)

None


In [14]:
datasets[0][0].keys()

2024-03-19 17:49:10,451 - INFO - finding looplift candidates


dict_keys(['metadata', 'points', 'voxels', 'shape', 'num_points', 'num_voxels', 'coordinates', 'gt_boxes_and_cls', 'hm', 'anno_box', 'ind', 'mask', 'cat'])

In [15]:
datasets[0][0]['metadata']

{'image_prefix': PosixPath('data/nuScenes'),
 'num_point_features': 5,
 'token': '0315ee314b674e7d9a3bc6d39ed61d64'}

In [16]:
"""
train_detector(
        model,
        datasets,
        cfg,
        distributed=distributed,
        validate=args.validate,
        logger=logger,
    )
"""

'\ntrain_detector(\n        model,\n        datasets,\n        cfg,\n        distributed=distributed,\n        validate=args.validate,\n        logger=logger,\n    )\n'

## train detector

In [17]:
from torch.utils.data import DataLoader
from det3d.datasets import DATASETS, build_dataloader
# cfg.data.samples_per_gpu: batch_size
# cfg.data.workers_per_gpu: workers_per_gpu, num_workers


data_loaders = [
        build_dataloader(
            ds, cfg.data.samples_per_gpu, cfg.data.workers_per_gpu, dist=distributed
        )
        for ds in datasets
    ]

In [18]:
data_loaders[0]

In [19]:
logger.info(f"total epochs: {cfg.total_epochs}" )
logger.info(f" length of dataloader elems: {len(data_loaders[0])}")

total_steps = cfg.total_epochs * len(data_loaders[0])
logger.info(f"total_steps: {total_steps}")

2024-03-19 17:49:12,867 - INFO - total epochs: 20
2024-03-19 17:49:12,868 - INFO -  length of dataloader elems: 30895
2024-03-19 17:49:12,868 - INFO - total_steps: 617900


In [20]:
cfg.lr_config

{'type': 'one_cycle',
 'lr_max': 0.001,
 'moms': [0.95, 0.85],
 'div_factor': 10.0,
 'pct_start': 0.4}

In [21]:
from functools import partial
from det3d.solver.fastai_optim import OptimWrapper
from det3d.builder import _create_learning_rate_scheduler
from torch import nn


def flatten_model(m):
    return sum(map(flatten_model, m.children()), []) if len(list(m.children())) else [m]

def get_layer_groups(m):
    return [nn.Sequential(*flatten_model(m))]

def build_one_cycle_optimizer(model, optimizer_config):
    if optimizer_config.fixed_wd:
        optimizer_func = partial(
            torch.optim.Adam, betas=(0.9, 0.99), amsgrad=optimizer_config.amsgrad
        )


    optimizer = OptimWrapper.create(
        optimizer_func,
        3e-3,   # TODO: CHECKING LR HERE !!!
        get_layer_groups(model),
        wd=optimizer_config.wd,
        true_wd=optimizer_config.fixed_wd,
        bn_wd=True,
    )

    return optimizer

if cfg.lr_config.type == "one_cycle":
    # build trainer
    optimizer = build_one_cycle_optimizer(model, cfg.optimizer)
    lr_scheduler = _create_learning_rate_scheduler(
        optimizer, cfg.lr_config, total_steps
    )
    cfg.lr_config = None

In [22]:
model = model.cuda()
logger.info(f"model structure: {model}")

2024-03-19 17:49:13,814 - INFO - model structure: PointPillars(
  (reader): PillarFeatureNet(
    (pfn_layers): ModuleList(
      (0): PFNLayer(
        (linear): Linear(in_features=10, out_features=32, bias=False)
        (norm): BatchNorm1d(32, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
      )
      (1): PFNLayer(
        (linear): Linear(in_features=64, out_features=64, bias=False)
        (norm): BatchNorm1d(64, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
      )
    )
  )
  (backbone): PointPillarsScatter()
  (neck): RPN(
    (blocks): ModuleList(
      (0): Sequential(
        (0): ZeroPad2d((1, 1, 1, 1))
        (1): Conv2d(64, 64, kernel_size=(3, 3), stride=(2, 2), bias=False)
        (2): BatchNorm2d(64, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
        (3): ReLU()
        (4): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (5): BatchNorm2d(64, eps=0.001, momentum=0.01, af

In [23]:
from det3d.torchie.trainer import DistSamplerSeedHook, Trainer, obj_from_dict
from collections import OrderedDict

def example_to_device(example, device, non_blocking=False) -> dict:
    example_torch = {}
    float_names = ["voxels", "bev_map"]
    for k, v in example.items():
        if k in ["anchors", "anchors_mask", "reg_targets", "reg_weights", "labels", "hm",
                "anno_box", "ind", "mask", 'cat', 'points']:
            example_torch[k] = [res.to(device, non_blocking=non_blocking) for res in v]
        elif k in [
            "voxels",
            "bev_map",
            "coordinates",
            "num_points",
            "num_voxels",
            "cyv_voxels",
            "cyv_num_voxels",
            "cyv_coordinates",
            "cyv_num_points",
            "gt_boxes_and_cls"
        ]:
            example_torch[k] = v.to(device, non_blocking=non_blocking)
        elif k == "calib":
            calib = {}
            for k1, v1 in v.items():
                calib[k1] = v1.to(device, non_blocking=non_blocking)
            example_torch[k] = calib
        else:
            example_torch[k] = v

    return example_torch


def parse_second_losses(losses):

    log_vars = OrderedDict()
    loss = sum(losses["loss"])
    for loss_name, loss_value in losses.items():
        if loss_name == "loc_loss_elem":
            log_vars[loss_name] = [[i.item() for i in j] for j in loss_value]
        else:
            log_vars[loss_name] = [i.item() for i in loss_value]

    return loss, log_vars

def batch_processor(model, data, train_mode, **kwargs):

    if "local_rank" in kwargs:
        device = torch.device(kwargs["local_rank"])
    else:
        device = None

    # data = example_convert_to_torch(data, device=device)
    example = example_to_device(data, device, non_blocking=False)

    del data

    if train_mode:
        losses = model(example, return_loss=True)
        loss, log_vars = parse_second_losses(losses)

        outputs = dict(
            loss=loss, log_vars=log_vars, num_samples=len(example["anchors"][0])
        )
        return outputs
    else:
        return model(example, return_loss=False)


trainer = Trainer(
        model, batch_processor, optimizer, lr_scheduler, cfg.work_dir, cfg.log_level
    )

In [24]:
logger.info(f"optimizer_config: {cfg.optimizer_config}")
optimizer_config = cfg.optimizer_config
trainer.register_training_hooks(
        cfg.lr_config, optimizer_config, cfg.checkpoint_config, cfg.log_config
    )


2024-03-19 17:49:13,846 - INFO - optimizer_config: {'grad_clip': {'max_norm': 35, 'norm_type': 2}}


## trainer.run

In [25]:
# trainer.run(data_loaders, cfg.workflow, cfg.total_epochs, local_rank=cfg.local_rank)

In [26]:
# trainer.run 파라미터
logger.info(f"num of dataloader: {len(data_loaders)}, dataloader type: {type(data_loaders[0])}")
logger.info(f"workflow information: {cfg.workflow}")
logger.info(f"total_epochs: {cfg.total_epochs}")
logger.info(f"local_rank: {cfg.local_rank}")

2024-03-19 17:49:13,883 - INFO - num of dataloader: 1, dataloader type: <class 'torch.utils.data.dataloader.DataLoader'>
2024-03-19 17:49:13,884 - INFO - workflow information: [('train', 1)]
2024-03-19 17:49:13,885 - INFO - total_epochs: 20
2024-03-19 17:49:13,886 - INFO - local_rank: 0


In [27]:
_max_epochs = cfg.total_epochs
work_dir = '/home/jaelee/objdect/CenterPoint/work_dirs/nusc_centerpoint_voxelnet_0075voxel_fix_bn_z'
workflow = cfg.workflow

In [28]:
idx = 0
mode, epochs = workflow[idx]
logger.info(f"mode: {mode}")
logger.info(f"epochs: {epochs}")

2024-03-19 17:49:13,922 - INFO - mode: train
2024-03-19 17:49:13,924 - INFO - epochs: 1


## train

In [29]:
# train(data_loaders[idx], epochs, **kwargs)

In [30]:
def batch_processor_inline(self, model, data, train_mode, **kwargs):

    if "local_rank" in kwargs:
        device = torch.device(kwargs["local_rank"])
    else:
        device = None

    # data = example_convert_to_torch(data, device=device)
    example = example_to_device(
        data, torch.cuda.current_device(), non_blocking=False
    )

    self.call_hook("after_data_to_device")

    if train_mode:
        losses = model(example, return_loss=True)
        self.call_hook("after_forward")
        loss, log_vars = parse_second_losses(losses)
        del losses

        outputs = dict(
            loss=loss, log_vars=log_vars, num_samples=-1  # TODO: FIX THIS
        )
        self.call_hook("after_parse_loss")

        return outputs
    else:
        return model(example, return_loss=False)

model.train()
mode = "train"
data_loader = data_loaders[idx]
length = len(data_loader)
_max_iters = _max_epochs * length
base_step = epochs * length
# outputs = batch_processor_inline(model, data_loader, train_mode=True, local_rank = 0)

### batch_processor_inline

In [31]:
device = torch.device(0)
lst = []
for i, data_batch in enumerate(data_loader):
    s = time.time()
    global_step = base_step + i
    example = example_to_device(
    data_batch, device, non_blocking=False)
    model(example, return_loss=True)
    # losses = model(example, return_loss=True)
    # loss, log_vars = parse_second_losses(losses)
    # del losses
    # outputs = dict(
    #             loss=loss, log_vars=log_vars, num_samples=-1  # TODO: FIX THIS
    #         )
    e = time.time() - s
    print(f"model train time: {e}")
    lst.append(e)
    if i == 30:
        break
lst = lst[1:]
avg = sum(lst[1:])/i
print(f"{i} samples avg train time: {sum(lst)/(i+1)}")
var = sum((np.array(lst) - avg)**2)/i
print(f"{i} samples var: {var}")
print(f"max: {max(lst)}")

model train time: 3.6843740940093994
model train time: 0.06357216835021973
model train time: 0.06265878677368164
model train time: 0.0651092529296875
model train time: 0.06926798820495605
model train time: 0.10083937644958496
model train time: 0.11492466926574707
model train time: 0.12358212471008301
model train time: 0.07884573936462402
model train time: 0.07666659355163574
model train time: 0.08623576164245605
model train time: 0.09369921684265137
model train time: 0.07455992698669434
model train time: 0.08440756797790527
model train time: 0.06857919692993164
model train time: 0.07120943069458008
model train time: 0.11033821105957031
model train time: 0.11503076553344727
model train time: 0.07001638412475586
model train time: 0.07343101501464844
model train time: 0.08123040199279785
model train time: 0.06880617141723633
model train time: 0.18772554397583008
model train time: 0.16376423835754395
model train time: 0.12022852897644043
model train time: 0.1419527530670166
model train tim

In [32]:
print(f"max: {max(lst)}")

max: 0.18772554397583008


In [ ]:
example

{'metadata': [{'image_prefix': PosixPath('data/nuScenes'),
   'num_point_features': 5,
   'token': '507b20e2538a4304aee0225dcf75e6fe'},
  {'image_prefix': PosixPath('data/nuScenes'),
   'num_point_features': 5,
   'token': 'bf6328d5bea742e2a2cacbc73155fcc9'},
  {'image_prefix': PosixPath('data/nuScenes'),
   'num_point_features': 5,
   'token': 'ba339102244045648e36362806cd6a8c'},
  {'image_prefix': PosixPath('data/nuScenes'),
   'num_point_features': 5,
   'token': '2bfb1fe8dd6a43b2a66ce769d763fb9e'}],
 'points': [tensor([[-5.2653e-05,  4.5746e-05,  1.5444e-05,  6.7000e+01,  0.0000e+00],
          [ 2.1557e+00,  4.1172e+00, -1.6914e+00,  1.0000e+02,  4.9717e-02],
          [-4.6112e+00,  2.8979e+00, -1.9824e+00,  1.1000e+01,  0.0000e+00],
          ...,
          [ 2.8922e+00,  5.2524e-01, -3.4294e-01,  2.0000e+00,  4.4954e-01],
          [-3.5513e+00, -1.6675e+00, -2.0864e+00,  0.0000e+00,  4.4954e-01],
          [ 2.3404e+00, -1.0537e+00, -5.4535e-01,  5.0000e+00,  2.9970e-01]],
   

In [ ]:
for k, v in example.items():
    print(f"{k}")

metadata
points
voxels
shape
num_points
num_voxels
coordinates
gt_boxes_and_cls
hm
anno_box
ind
mask
cat


## model

In [ ]:
# model = build_detector(cfg.model, train_cfg=cfg.train_cfg, test_cfg=cfg.test_cfg)
logger.info(f"model: {cfg.model}")
logger.info(f"train_cfg: {cfg.train_cfg}")
logger.info(f"test_cfg: {cfg.test_cfg}")
logger.info(f"{dict(train_cfg=cfg.train_cfg, test_cfg=cfg.test_cfg)}")
default_args = dict(train_cfg=cfg.train_cfg, test_cfg=cfg.test_cfg)
# def build_detector(cfg, train_cfg=None, test_cfg=None):
#    return build(cfg, DETECTORS, dict(train_cfg=train_cfg, test_cfg=test_cfg))

2024-03-14 11:34:34,739 - INFO - model: {'type': 'PointPillars', 'pretrained': None, 'reader': {'type': 'PillarFeatureNet', 'num_filters': [64, 64], 'num_input_features': 5, 'with_distance': False, 'voxel_size': (0.2, 0.2, 8), 'pc_range': (-51.2, -51.2, -5.0, 51.2, 51.2, 3.0)}, 'backbone': {'type': 'PointPillarsScatter', 'ds_factor': 1}, 'neck': {'type': 'RPN', 'layer_nums': [3, 5, 5], 'ds_layer_strides': [2, 2, 2], 'ds_num_filters': [64, 128, 256], 'us_layer_strides': [0.5, 1, 2], 'us_num_filters': [128, 128, 128], 'num_input_features': 64, 'logger': <Logger RPN (INFO)>}, 'bbox_head': {'type': 'CenterHead', 'in_channels': 384, 'tasks': [{'num_class': 1, 'class_names': ['car']}, {'num_class': 2, 'class_names': ['truck', 'construction_vehicle']}, {'num_class': 2, 'class_names': ['bus', 'trailer']}, {'num_class': 1, 'class_names': ['barrier']}, {'num_class': 2, 'class_names': ['motorcycle', 'bicycle']}, {'num_class': 2, 'class_names': ['pedestrian', 'traffic_cone']}], 'dataset': 'nuscene

In [ ]:
type(cfg.model)

det3d.torchie.utils.config.ConfigDict

In [ ]:
cfg.model

{'type': 'PointPillars',
 'pretrained': None,
 'reader': {'type': 'PillarFeatureNet',
  'num_filters': [64, 64],
  'num_input_features': 5,
  'with_distance': False,
  'voxel_size': (0.2, 0.2, 8),
  'pc_range': (-51.2, -51.2, -5.0, 51.2, 51.2, 3.0)},
 'backbone': {'type': 'PointPillarsScatter', 'ds_factor': 1},
 'neck': {'type': 'RPN',
  'layer_nums': [3, 5, 5],
  'ds_layer_strides': [2, 2, 2],
  'ds_num_filters': [64, 128, 256],
  'us_layer_strides': [0.5, 1, 2],
  'us_num_filters': [128, 128, 128],
  'num_input_features': 64,
  'logger': <Logger RPN (INFO)>},
 'bbox_head': {'type': 'CenterHead',
  'in_channels': 384,
  'tasks': [{'num_class': 1, 'class_names': ['car']},
   {'num_class': 2, 'class_names': ['truck', 'construction_vehicle']},
   {'num_class': 2, 'class_names': ['bus', 'trailer']},
   {'num_class': 1, 'class_names': ['barrier']},
   {'num_class': 2, 'class_names': ['motorcycle', 'bicycle']},
   {'num_class': 2, 'class_names': ['pedestrian', 'traffic_cone']}],
  'dataset'

In [ ]:
len(cfg.model)

6

In [ ]:
args = cfg.model.copy()
print(args)
obj_type = args.pop("type")
print(obj_type)
if default_args is not None:
    for name, value in default_args.items():
        args.setdefault(name, value)

{'type': 'PointPillars', 'pretrained': None, 'reader': {'type': 'PillarFeatureNet', 'num_filters': [64, 64], 'num_input_features': 5, 'with_distance': False, 'voxel_size': (0.2, 0.2, 8), 'pc_range': (-51.2, -51.2, -5.0, 51.2, 51.2, 3.0)}, 'backbone': {'type': 'PointPillarsScatter', 'ds_factor': 1}, 'neck': {'type': 'RPN', 'layer_nums': [3, 5, 5], 'ds_layer_strides': [2, 2, 2], 'ds_num_filters': [64, 128, 256], 'us_layer_strides': [0.5, 1, 2], 'us_num_filters': [128, 128, 128], 'num_input_features': 64, 'logger': <Logger RPN (INFO)>}, 'bbox_head': {'type': 'CenterHead', 'in_channels': 384, 'tasks': [{'num_class': 1, 'class_names': ['car']}, {'num_class': 2, 'class_names': ['truck', 'construction_vehicle']}, {'num_class': 2, 'class_names': ['bus', 'trailer']}, {'num_class': 1, 'class_names': ['barrier']}, {'num_class': 2, 'class_names': ['motorcycle', 'bicycle']}, {'num_class': 2, 'class_names': ['pedestrian', 'traffic_cone']}], 'dataset': 'nuscenes', 'weight': 0.25, 'code_weights': [1.0

In [ ]:
for k, v in args.items():
    print(f"{k} : {v}")

pretrained : None
reader : {'type': 'PillarFeatureNet', 'num_filters': [64, 64], 'num_input_features': 5, 'with_distance': False, 'voxel_size': (0.2, 0.2, 8), 'pc_range': (-51.2, -51.2, -5.0, 51.2, 51.2, 3.0)}
backbone : {'type': 'PointPillarsScatter', 'ds_factor': 1}
neck : {'type': 'RPN', 'layer_nums': [3, 5, 5], 'ds_layer_strides': [2, 2, 2], 'ds_num_filters': [64, 128, 256], 'us_layer_strides': [0.5, 1, 2], 'us_num_filters': [128, 128, 128], 'num_input_features': 64, 'logger': <Logger RPN (INFO)>}
bbox_head : {'type': 'CenterHead', 'in_channels': 384, 'tasks': [{'num_class': 1, 'class_names': ['car']}, {'num_class': 2, 'class_names': ['truck', 'construction_vehicle']}, {'num_class': 2, 'class_names': ['bus', 'trailer']}, {'num_class': 1, 'class_names': ['barrier']}, {'num_class': 2, 'class_names': ['motorcycle', 'bicycle']}, {'num_class': 2, 'class_names': ['pedestrian', 'traffic_cone']}], 'dataset': 'nuscenes', 'weight': 0.25, 'code_weights': [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.2, 0.

In [ ]:
from det3d.utils import Registry
from det3d.models.registry import (
    BACKBONES,
    DETECTORS,
    HEADS,
    LOSSES,
    NECKS,
    READERS,
    SECOND_STAGE,
    ROI_HEAD
)


In [ ]:
DETECTORS

Registry(name=detector, items=['SingleStageDetector', 'PointPillars', 'VoxelNet', 'TwoStageDetector'])

In [ ]:
obj_cls = DETECTORS.get(obj_type)
type(obj_cls)

type

In [ ]:
obj_cls

det3d.models.detectors.point_pillars.PointPillars

In [ ]:
args.keys()

dict_keys(['pretrained', 'reader', 'backbone', 'neck', 'bbox_head', 'train_cfg', 'test_cfg'])

In [ ]:
obj_cls(**args)

2024-03-14 11:35:00,417 - INFO - Finish RPN Initialization
2024-03-14 11:35:00,418 - INFO - num_classes: [1, 2, 2, 1, 2, 2]
2024-03-14 11:35:00,437 - INFO - Finish CenterHead Initialization


Use HM Bias:  -2.19


PointPillars(
  (reader): PillarFeatureNet(
    (pfn_layers): ModuleList(
      (0): PFNLayer(
        (linear): Linear(in_features=10, out_features=32, bias=False)
        (norm): BatchNorm1d(32, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
      )
      (1): PFNLayer(
        (linear): Linear(in_features=64, out_features=64, bias=False)
        (norm): BatchNorm1d(64, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
      )
    )
  )
  (backbone): PointPillarsScatter()
  (neck): RPN(
    (blocks): ModuleList(
      (0): Sequential(
        (0): ZeroPad2d((1, 1, 1, 1))
        (1): Conv2d(64, 64, kernel_size=(3, 3), stride=(2, 2), bias=False)
        (2): BatchNorm2d(64, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
        (3): ReLU()
        (4): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (5): BatchNorm2d(64, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
        (6): 

In [ ]:
example

{'metadata': [{'image_prefix': PosixPath('data/nuScenes'),
   'num_point_features': 5,
   'token': '507b20e2538a4304aee0225dcf75e6fe'},
  {'image_prefix': PosixPath('data/nuScenes'),
   'num_point_features': 5,
   'token': 'bf6328d5bea742e2a2cacbc73155fcc9'},
  {'image_prefix': PosixPath('data/nuScenes'),
   'num_point_features': 5,
   'token': 'ba339102244045648e36362806cd6a8c'},
  {'image_prefix': PosixPath('data/nuScenes'),
   'num_point_features': 5,
   'token': '2bfb1fe8dd6a43b2a66ce769d763fb9e'}],
 'points': [tensor([[-5.2653e-05,  4.5746e-05,  1.5444e-05,  6.7000e+01,  0.0000e+00],
          [ 2.1557e+00,  4.1172e+00, -1.6914e+00,  1.0000e+02,  4.9717e-02],
          [-4.6112e+00,  2.8979e+00, -1.9824e+00,  1.1000e+01,  0.0000e+00],
          ...,
          [ 2.8922e+00,  5.2524e-01, -3.4294e-01,  2.0000e+00,  4.4954e-01],
          [-3.5513e+00, -1.6675e+00, -2.0864e+00,  0.0000e+00,  4.4954e-01],
          [ 2.3404e+00, -1.0537e+00, -5.4535e-01,  5.0000e+00,  2.9970e-01]],
   

In [ ]:
for k, v in example.items():
    print(f"{k} : {v}")

metadata : [{'image_prefix': PosixPath('data/nuScenes'), 'num_point_features': 5, 'token': '507b20e2538a4304aee0225dcf75e6fe'}, {'image_prefix': PosixPath('data/nuScenes'), 'num_point_features': 5, 'token': 'bf6328d5bea742e2a2cacbc73155fcc9'}, {'image_prefix': PosixPath('data/nuScenes'), 'num_point_features': 5, 'token': 'ba339102244045648e36362806cd6a8c'}, {'image_prefix': PosixPath('data/nuScenes'), 'num_point_features': 5, 'token': '2bfb1fe8dd6a43b2a66ce769d763fb9e'}]
points : [tensor([[-5.2653e-05,  4.5746e-05,  1.5444e-05,  6.7000e+01,  0.0000e+00],
        [ 2.1557e+00,  4.1172e+00, -1.6914e+00,  1.0000e+02,  4.9717e-02],
        [-4.6112e+00,  2.8979e+00, -1.9824e+00,  1.1000e+01,  0.0000e+00],
        ...,
        [ 2.8922e+00,  5.2524e-01, -3.4294e-01,  2.0000e+00,  4.4954e-01],
        [-3.5513e+00, -1.6675e+00, -2.0864e+00,  0.0000e+00,  4.4954e-01],
        [ 2.3404e+00, -1.0537e+00, -5.4535e-01,  5.0000e+00,  2.9970e-01]],
       device='cuda:0'), tensor([[ -1.9059,   4.73

In [ ]:
voxels = example["voxels"]
coordinates = example["coordinates"]
num_points_in_voxel = example["num_points"]
num_voxels = example["num_voxels"]
batch_size = len(num_voxels)
data = dict(
    features=voxels,
    num_voxels=num_points_in_voxel,
    coors=coordinates,
    batch_size=batch_size,
    input_shape=example["shape"][0],
)

In [ ]:
voxels.shape

torch.Size([76275, 20, 5])

In [ ]:
data

{'features': tensor([[[-7.3953e+00, -6.7586e+00, -1.2526e+00,  1.0000e+02,  1.4983e-01],
          [-7.4089e+00, -6.7874e+00, -1.2638e+00,  1.0000e+02,  1.9960e-01],
          [-7.3710e+00, -6.8147e+00, -1.2546e+00,  1.0000e+02,  0.0000e+00],
          ...,
          [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
          [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
          [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00]],
 
         [[-5.8084e+00, -1.0606e+01, -9.7923e-01,  3.2000e+01,  1.4983e-01],
          [-5.8278e+00, -1.0621e+01, -9.9175e-01,  2.1000e+01,  3.9980e-01],
          [-5.8144e+00, -1.0608e+01, -9.9364e-01,  2.4000e+01,  3.4947e-01],
          ...,
          [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
          [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
          [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00]],
 
         [[-8.3508e+00, -3.5

In [ ]:
READERS

Registry(name=reader, items=['PillarFeatureNet', 'VoxelFeatureExtractorV3', 'DynamicVoxelEncoder'])

In [ ]:
input_features = self.reader(data["features"], data['num_voxels'])

NameError: name 'self' is not defined